In [1]:
import random

random.seed(23)


class Card:
    def __init__(self, value):
        self.value = value % 8
        self.suit = value // 8
        self.isTrump = False
        colors = ["♠", "♥", "♦", "♣"]
        values = ["7", "8", "9", "J", "Q", "K", "10", "A"]
        self.repr = colors[self.suit] + values[self.value]

    def set_trump(self):
        self.isTrump = True
        value_changes = [0, 1, 6, 7, 2, 3, 4, 5]
        self.value = value_changes[self.value]
        self.value += 10

    def __repr__(self):
        return str((self.repr, self.value))


class Deck:
    def __init__(self):
        self.cards = [Card(i) for i in range(32)]
        random.shuffle(self.cards)

    def deal(self) -> list:
        assert len(self.cards) == 32
        shuffled = random.sample(self.cards.copy(), 32)
        return [shuffled[i : i + 8] for i in range(0, 32, 8)]

    def remove_trick(self, trick):
        for card in trick:
            self.cards.remove(card)


class Player:
    def __init__(self, hand):
        self.hand = hand
        self.trump = None

    def sort_hand(self):
        self.hand.sort(key=lambda x: -x.value)

    def define_trump(self, trump: int):
        self.trump = trump
        for card in self.hand:
            if card.suit == trump:
                card.set_trump()
            self.sort_hand()

    def is_leading_card(self, current_trick, leading_suit):
        if not current_trick:
            return True
        elif len(current_trick) == 1:
            return False
        else:
            partner_card = current_trick[-2]
            opponent_cards = current_trick[-1::-2]
            if partner_card.suit == leading_suit:
                if self.trump in [card.suit for card in opponent_cards]:
                    return False
            else:
                return False

    def get_legal_cards(self, current_trick: list):
        if not current_trick:
            return self.hand

        else:
            leading_suit = current_trick[0].suit
            if any(card.suit == leading_suit for card in self.hand):
                return [card for card in self.hand if card.suit == leading_suit]
            # elif len(current_trick) > 1 and current_trick[-2].suit == self.trump:
            #     return self.hand
            elif self.trump in [card.suit for card in self.hand]:
                return [card for card in self.hand if card.suit == self.trump]
            else:
                return self.hand

    def play(self, passed_tricks, current_trick):
        """given a game state return a card"""
        raise NotImplementedError


class CoincheGame:
    def __init__(self, players):
        self.deck = Deck()
        hands = self.deck.deal()
        self.players = [players[i](hands[i]) for i in range(4)]
        self.tricks = []

        # TODO Ici les annonces

        self.trump = 0  # atout pique default
        for player in self.players:
            player.define_trump(self.trump)

    def play(self):
        for i in range(8):

            print(*[p.hand for p in self.players], sep="\n")

            current_trick = []
            for player in self.players:
                card = player.play(self.tricks, current_trick)
                current_trick.append(card)

            print(f"{current_trick = }")
            print("-" * 30)

            self.tricks.append(current_trick)


class RandomPlayer(Player):
    def __init__(self, hand):
        super().__init__(hand)

    def play(self, passed_tricks, current_trick):
        legal_cards = self.get_legal_cards(current_trick)
        chosen_card = random.choice(legal_cards)
        self.hand.remove(chosen_card)
        return chosen_card

In [2]:
players = [RandomPlayer, RandomPlayer, RandomPlayer, RandomPlayer]
game = CoincheGame(players)
# game.play()

print(*[game.players[i].hand for i in range(4)], sep="\n")

[('♠K', 13), ('♠8', 11), ('♠10', 6), ('♦Q', 4), ('♥Q', 4), ('♥J', 3), ('♦8', 1), ('♥7', 0)]
[('♠A', 7), ('♥K', 5), ('♦K', 5), ('♣Q', 4), ('♣9', 2), ('♦9', 2), ('♦7', 0), ('♣7', 0)]
[('♠9', 16), ('♠Q', 12), ('♠7', 10), ('♦A', 7), ('♣A', 7), ('♥A', 7), ('♥10', 6), ('♦10', 6)]
[('♠J', 17), ('♣10', 6), ('♣K', 5), ('♦J', 3), ('♣J', 3), ('♥9', 2), ('♣8', 1), ('♥8', 1)]
